## Preliminary experimental workflow to register multi-view light sheet data

Notes:
- registration: so far only translation registration is performed
- fusion: only vanilla linear blending currently supported
- generally
  - this is a first hacky workflow that will strongly change in API (and get simplified)
  - documentation will follow

In [1]:
# imports

import os
import numpy as np
from pathlib import Path
from tqdm import tqdm
import dask.diagnostics, tempfile

import napari

from napari_stitcher import _reader
from napari_stitcher import _msi_utils, _spatial_image_utils
from napari_stitcher import _viewer_utils
from napari_stitcher import _registration
from napari_stitcher import _fusion

import tifffile
%matplotlib widget

### Load data into multiscale zarr files (close to NGFF format)

In [2]:
base_dir = '/Users/malbert/software/napari-stitcher/image-datasets/multi-view/old_mDSLM_classical_4_angles_10x_0.3NA_detection'
filenames = [(os.path.join(base_dir, f)) for f in os.listdir(base_dir) if f.endswith('.tif')]

# sort angles
filenames = [Path(fn) for fn in sorted(filenames)]
print('Files:')
print('\n'.join([fn.name for fn in filenames]))


Files:
MGolden2022A-DS0031TP0001DR0001CH0001PL(ZS).tif
MGolden2022A-DS0031TP0001DR0002CH0001PL(ZS).tif
MGolden2022A-DS0031TP0001DR0003CH0001PL(ZS).tif
MGolden2022A-DS0031TP0001DR0004CH0001PL(ZS).tif


In [3]:
import importlib

_reader = importlib.reload(_reader)

msims = []
for filename in tqdm(filenames):
    msim = _msi_utils.get_store_decorator(
        filename.with_suffix('.zarr'),
        store_overwrite=False)(
            _msi_utils.get_msim_from_xim)(
                _reader.read_tiff_into_spatial_xarray(
                    filename,
                    scale={'z': 2.58, 'y': 0.645, 'x': 0.645}
                    
                ))
    msims.append(msim)


100%|█| 4/4 [00:00<00:00,  9.00


### Set estimate of initial transformations

In [4]:
import transformations as tf

_msi_utils = importlib.reload(_msi_utils)

for imsim, msim in enumerate(msims):

    affine = tf.rotation_matrix(
        -np.pi/2 * imsim,
        point=_spatial_image_utils.get_center_of_xim(msims[imsim]['scale0/image']),#, transform="affine_metadata"),
        direction=[0,0,1],
        )
    
    _msi_utils.set_affine_transform(
        msim,
        affine[None], # one tp
        'affine_metadata'
    )


### Visualize pre-registered views

In [5]:
viewer = napari.Viewer(ndisplay=3)
lds = _viewer_utils.create_image_layer_tuples_from_msims(msims, transform_key='affine_metadata', n_colors=4)

_viewer_utils.add_image_layer_tuples_to_viewer(viewer, lds)

/Users/malbert/miniconda3/envs/napari-stitcher/lib/python3.10/site-packages/napari/layers/image/image.py:761: UserWarning: Multiscale rendering is only supported in 2D. In 3D, only the lowest resolution scale is displayed
  warnings.warn(
/Users/malbert/miniconda3/envs/napari-stitcher/lib/python3.10/site-packages/napari/layers/image/image.py:761: UserWarning: Multiscale rendering is only supported in 2D. In 3D, only the lowest resolution scale is displayed
  warnings.warn(
/Users/malbert/miniconda3/envs/napari-stitcher/lib/python3.10/site-packages/napari/layers/utils/_slice_input.py:72: UserWarning: Non-orthogonal slicing is being requested, but is not fully supported. Data is displayed without applying an out-of-slice rotation or shear component.
  warnings.warn(
/Users/malbert/miniconda3/envs/napari-stitcher/lib/python3.10/site-packages/napari/layers/image/image.py:761: UserWarning: Multiscale rendering is only supported in 2D. In 3D, only the lowest resolution scale is displayed
  w

[<Image layer 'tile_000 :: 0' at 0x29f82e2f0>,
 <Image layer 'tile_001 :: 0' at 0x29f885480>,
 <Image layer 'tile_002 :: 0' at 0x2b0688070>,
 <Image layer 'tile_003 :: 0' at 0x2b0545690>]

### Register views

In [10]:
import importlib
_registration = importlib.reload(_registration)

with dask.diagnostics.ProgressBar():
    params = _registration.register(
        [_msi_utils.get_xim_from_msim(msim) for msim in msims],
        registration_binning={'z': 2, 'y': 8, 'x': 8},
        reg_channel_index=0,
        transform_key='affine_metadata',
    )
    
for msim, param in zip(msims, params):
    _msi_utils.set_affine_transform(msim, param, transform_key='affine_registered', base_transform_key='affine_metadata')

[####################################### ] | 99% Completed | 3.06 s ms

/Users/malbert/miniconda3/envs/napari-stitcher/lib/python3.10/site-packages/numpy/lib/function_base.py:2853: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/Users/malbert/miniconda3/envs/napari-stitcher/lib/python3.10/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


[########################################] | 100% Completed | 3.69 s


### Visualize registration

In [11]:
viewer = napari.Viewer(ndisplay=3)

lds = _viewer_utils.create_image_layer_tuples_from_msims(
    msims, transform_key='affine_metadata', n_colors=4,
    name_prefix='pre-registered view')
mlayers = _viewer_utils.add_image_layer_tuples_to_viewer(viewer, lds, do_link_layers=True)

lds = _viewer_utils.create_image_layer_tuples_from_msims(
    msims, transform_key='affine_registered', n_colors=4,
    name_prefix='registered view')
rlayers = _viewer_utils.add_image_layer_tuples_to_viewer(viewer, lds, do_link_layers=True)

/Users/malbert/miniconda3/envs/napari-stitcher/lib/python3.10/site-packages/napari/layers/image/image.py:761: UserWarning: Multiscale rendering is only supported in 2D. In 3D, only the lowest resolution scale is displayed
  warnings.warn(
/Users/malbert/miniconda3/envs/napari-stitcher/lib/python3.10/site-packages/napari/layers/image/image.py:761: UserWarning: Multiscale rendering is only supported in 2D. In 3D, only the lowest resolution scale is displayed
  warnings.warn(
/Users/malbert/miniconda3/envs/napari-stitcher/lib/python3.10/site-packages/napari/layers/utils/_slice_input.py:72: UserWarning: Non-orthogonal slicing is being requested, but is not fully supported. Data is displayed without applying an out-of-slice rotation or shear component.
  warnings.warn(
/Users/malbert/miniconda3/envs/napari-stitcher/lib/python3.10/site-packages/napari/layers/image/image.py:761: UserWarning: Multiscale rendering is only supported in 2D. In 3D, only the lowest resolution scale is displayed
  w

### Fuse views (linear blending)

In [13]:
import importlib
_fusion = importlib.reload(_fusion)
from napari_stitcher import _transformation
_transformation = importlib.reload(_transformation)

xims = [_msi_utils.get_xim_from_msim(msim) for msim in msims]

tmpdir = tempfile.TemporaryDirectory()

fused = _fusion.fuse(
    xims[:],
    transform_key='affine_registered',
    output_spacing={dim: 5. for dim in ['z', 'y', 'x']})#, tmpdir=tmpdir)

# WHY DOES IT COMPUTE TWICE?

mfused = _msi_utils.get_msim_from_xim(fused, scale_factors=None)

fused_path = os.path.join(tmpdir.name, 'fused.zarr')
with dask.diagnostics.ProgressBar():
    mfused.to_zarr(fused_path)
    
mfused = _msi_utils.multiscale_spatial_image_from_zarr(fused_path)

[########################################] | 100% Completed | 2.05 sms


### Visualize fusion in napari

In [14]:
viewer = napari.Viewer(ndisplay=3)

lds = _viewer_utils.create_image_layer_tuples_from_msims(
    msims, transform_key='affine_registered', n_colors=4,
    name_prefix='registered view')

rlayers = _viewer_utils.add_image_layer_tuples_to_viewer(viewer, lds, do_link_layers=True)

ld = _viewer_utils.create_image_layer_tuple_from_msim(mfused, transform_key='affine_registered', name_prefix='fused')

_viewer_utils.add_image_layer_tuples_to_viewer(viewer, [ld])

/Users/malbert/miniconda3/envs/napari-stitcher/lib/python3.10/site-packages/napari/layers/image/image.py:761: UserWarning: Multiscale rendering is only supported in 2D. In 3D, only the lowest resolution scale is displayed
  warnings.warn(
/Users/malbert/miniconda3/envs/napari-stitcher/lib/python3.10/site-packages/napari/layers/image/image.py:761: UserWarning: Multiscale rendering is only supported in 2D. In 3D, only the lowest resolution scale is displayed
  warnings.warn(
/Users/malbert/miniconda3/envs/napari-stitcher/lib/python3.10/site-packages/napari/layers/utils/_slice_input.py:72: UserWarning: Non-orthogonal slicing is being requested, but is not fully supported. Data is displayed without applying an out-of-slice rotation or shear component.
  warnings.warn(
/Users/malbert/miniconda3/envs/napari-stitcher/lib/python3.10/site-packages/napari/layers/image/image.py:761: UserWarning: Multiscale rendering is only supported in 2D. In 3D, only the lowest resolution scale is displayed
  w

[<Image layer 'fused :: [0]' at 0x15b16f0a0>]

In [15]:
# stream presaved fused image to tif

from napari_stitcher import _writer

import importlib
_writer = importlib.reload(_writer)

with dask.diagnostics.ProgressBar():
    _writer.save_xim_as_tif('fused.tif', _msi_utils.get_xim_from_msim(mfused))

[########################################] | 100% Completed | 106.38 ms
